In [2]:
!pip install -q sentence-transformers pandas scikit-learn

In [3]:
!pip install pandarallel rank_bm25

  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=5ad70726f380b79c62522787bad70e00d0ef5e784ca015b1399ddbd97186b7e0
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel


In [4]:
!pip install transformers --quiet



In [5]:
!pip install rank-bm25


In [6]:
from transformers import AutoTokenizer, AutoModel

model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
scibert = AutoModel.from_pretrained(model_name)
scibert.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm


In [8]:
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


In [9]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [10]:
PATH_COLLECTION_DATA = 'subtask4b_collection_data (2).pkl'
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [13]:
import pandas as pd
file_path = "subtask4b_query_tweets_train (1).tsv"
df_query_train = pd.read_csv(file_path, sep='\t')
df_query_train.head()


,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
PATH_QUERY_DEV_DATA = 'subtask4b_query_tweets_dev.tsv'
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

# Running the BM25 baseline

In [17]:
from rank_bm25 import BM25Okapi

In [18]:
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [19]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:100]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk

In [20]:
train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

if not os.path.exists(train_pkl_path):
    df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_train.to_pickle(train_pkl_path)
else:
    df_query_train = pd.read_pickle(train_pkl_path)

if not os.path.exists(dev_pkl_path):
    df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_dev.to_pickle(dev_pkl_path)
else:
    df_query_dev = pd.read_pickle(dev_pkl_path)

In [21]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [22]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')


print(f"Results on the train set: {dict((k, float(v)) for k, v in results_train.items())}")
print(f"Results on the dev set: {dict((k, float(v)) for k, v in results_dev.items())}")

Results on the train set: {1: 0.503959683225342, 5: 0.54762076176763, 10: 0.5524445799438086}
Results on the dev set: {1: 0.5057142857142857, 5: 0.5522738095238094, 10: 0.557658163265306}


# 1) import query and collection

In [23]:
df_query_train.head()


,post_id,tweet_text,cord_uid,bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,"[htlvpvz5, h7hj64q5, rwgqkow3, dbgtslc8, am11y...",1.000000
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,"[2cwvga0k, 33znyrn8, st3fyb64, 74dw6emg, b68c8...",0.000000
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"[8hkxbxz9, w98847ai, jtwb17u8, iy1enazk, pg0l9...",0.333333
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,"[32ua8wb6, zxe95qy9, 30pl5tx3, l4y7v729, b97ac...",0.000000
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,"[tiqksd69, k7smwz6w, b0dzhsrh, yc7cvbii, iiafu...",1.000000


In [24]:
df_query_dev.head()

,post_id,tweet_text,cord_uid,bm25_topk,in_topx
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80...",0.000000
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np...",1.000000
2,73,I recall early on reading that researchers who...,sts48u9i,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir...",0.166667
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt...",1.000000
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa...",0.125000


In [25]:
df_collection.head()


,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


# 3) Neural RE-Ranking

# 3.1) ColBERT w/ fine-tuned BERT

In [26]:
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    token_embeddings = token_embeddings[attention_mask]
    return token_embeddings


def build_and_save_doc_embeddings(
    docs_df,
    model_name,
    save_dir,
    max_len=512,
    device="cuda"
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    save_path = Path("doc_embeddings_" + save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata_path = save_path / "metadata.json"
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        metadata = {}

    print("Precomputing document embeddings.")
    for i, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_id = row.get("cord_uid", f"doc_{i}")
        file_path = save_path / f"{doc_id}.pt"

        if file_path.exists() and doc_id in metadata:
            continue

        text = str(row.get('title', '')) + " " + str(row.get('abstract', '')) + " Authors: " + str(row.get('authors', ''))

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        output = model(**inputs)
        token_embeddings = output.last_hidden_state.squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0).bool()
        token_embeddings = token_embeddings[attention_mask]

        n_tokens = token_embeddings.size(0)
        pad_len = max_len - n_tokens

        if pad_len > 0:
            padding = torch.zeros(pad_len, token_embeddings.size(1), device=device)
            token_embeddings = torch.cat([token_embeddings, padding], dim=0)
        else:
            token_embeddings = token_embeddings[:max_len]

        try:
            torch.save(token_embeddings.cuda(), file_path)
        except Exception as e:
            print(f"Error saving document {doc_id}: {e}")
            continue

        metadata[doc_id] = {
            "title": row.get("title", ""),
            "abstract": row.get("abstract", ""),
            "authors": row.get("authors", ""),
            "length": min(n_tokens, max_len),
            "path": str(file_path)
        }

    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    return metadata


def get_precomputed_doc_embeddings(save_name):
    def split_at_slash(s):
        if '/' in s:
            return s.split('/', 1)
        else:
            return ['', s]

    if not os.path.exists("doc_embeddings_" + split_at_slash(save_name)[1] + "/metadata.json"):
        metadata = build_and_save_doc_embeddings(df_collection, model_name=save_name, save_dir=save_name, device="cuda")
    else:
        with open("doc_embeddings_" + save_name + "/metadata.json", "r") as f:
            metadata = json.load(f)
    return metadata

In [27]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path

model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
scibert = AutoModel.from_pretrained(model_name)
scibert.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

# 3.2 MatchPyramid

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from tqdm import tqdm
import os



class MatchPyramid(nn.Module):
    def __init__(self, num_channels=8, kernel_size=3, mlp_hidden=64, dropout_rate=0.2):
        super(MatchPyramid, self).__init__()
        self.conv1 = nn.Conv2d(1, num_channels, kernel_size=kernel_size, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=kernel_size, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(num_channels, num_channels, kernel_size=kernel_size, padding=1)

        self.pool3 = nn.AdaptiveMaxPool2d((4, 4))

        self.adaptive_pool = nn.AdaptiveAvgPool2d((5, 5))

        self.fc1 = None
        self.mlp_hidden = mlp_hidden
        self.dropout_rate = dropout_rate
        self.fc2 = nn.Linear(mlp_hidden, 1)

    def forward(self, match_matrix):
        x = match_matrix.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)

        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), self.mlp_hidden).to(x.device)

        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        return self.fc2(x).squeeze(-1)



class MatchPyramidTripletDataset(Dataset):
    def __init__(self, df, metadata, num_negatives=2):
        self.data = []
        self.metadata = metadata

        for _, row in df.iterrows():
            query = row["tweet_text"]
            pos_id = row["cord_uid"]
            negatives = [doc for doc in row["bm25_topk"] if doc != pos_id]
            hard_negatives = negatives[:5]

            if hard_negatives:
                for _ in range(num_negatives):
                    neg_id = random.choice(hard_negatives)
                    self.data.append((query, pos_id, neg_id))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]



def create_match_matrix(query_embs, doc_embs, eps=1e-8):
    q_norm = query_embs / (query_embs.norm(dim=-1, keepdim=True) + eps)
    d_norm = doc_embs / (doc_embs.norm(dim=-1, keepdim=True) + eps)
    return torch.matmul(q_norm, d_norm.transpose(0, 1))



def train_matchpyramid(model, dataset, tokenizer, transformer_model, metadata,
                       margin=0.3, epochs=6, batch_size=8, lr=2e-4, verbose=True):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(DEVICE)
    transformer_model.to(DEVICE)
    transformer_model.eval()
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        batch_progress = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for batch_idx, batch in enumerate(batch_progress):
            queries, pos_ids, neg_ids = batch
            score_pos_list = []
            score_neg_list = []

            for i in range(len(queries)):
                try:

                    q_inputs = tokenizer(queries[i], return_tensors='pt', truncation=True, max_length=256).to(DEVICE)
                    outputs = transformer_model(**q_inputs)
                    q_embs = outputs.last_hidden_state.squeeze(0)
                    q_mask = q_inputs['attention_mask'].squeeze(0).bool()
                    q_embs = q_embs[q_mask]


                    pos_emb = torch.load(metadata[pos_ids[i]]["path"]).to(DEVICE)[:256]
                    neg_emb = torch.load(metadata[neg_ids[i]]["path"]).to(DEVICE)[:256]

                    if q_embs.size(0) < 8 or pos_emb.size(0) < 8 or neg_emb.size(0) < 8:
                        if verbose:
                            print(f"Skipped small embedding (idx {i})")
                        continue

                    if torch.isnan(q_embs).any() or torch.isnan(pos_emb).any() or torch.isnan(neg_emb).any():
                        if verbose:
                            print(f"NaNs in embeddings at batch {batch_idx}, sample {i}")
                        continue


                    m_pos = create_match_matrix(q_embs, pos_emb).unsqueeze(0)
                    m_neg = create_match_matrix(q_embs, neg_emb).unsqueeze(0)

                    if m_pos.size(-1) < 4 or m_pos.size(-2) < 4 or m_neg.size(-1) < 4 or m_neg.size(-2) < 4:
                        if verbose:
                            print(f"Skipped tiny match matrix at batch {batch_idx}, sample {i}")
                        continue


                    score_pos = model(m_pos)
                    score_neg = model(m_neg)

                    if torch.isnan(score_pos) or torch.isnan(score_neg):
                        if verbose:
                            print(f"NaNs in score at batch {batch_idx}, sample {i}")
                        continue

                    score_pos_list.append(score_pos)
                    score_neg_list.append(score_neg)

                except Exception as e:
                    if verbose:
                        print(f"Error at batch {batch_idx}, sample {i}: {e}")
                    continue

            if not score_pos_list or not score_neg_list:
                continue

            score_pos_batch = torch.stack(score_pos_list)
            score_neg_batch = torch.stack(score_neg_list)
            loss = F.relu(margin + score_neg_batch - score_pos_batch).mean()

            if torch.isnan(loss):
                print(f"NaN loss at batch {batch_idx}, skipping backward pass.")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        print(f" Epoch {epoch + 1}/{epochs} — Total Loss: {total_loss:.4f}")




In [53]:
import os

bad_metadata_path = "doc_embeddings_allenai/scibert_scivocab_uncased/metadata.json"
if os.path.exists(bad_metadata_path):
    os.remove(bad_metadata_path)
    print(" Corrupt metadata file removed.")
else:
    print(" File not found.")


 Corrupt metadata file removed.


In [54]:
import json

model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
scibert = AutoModel.from_pretrained(model_name)

In [55]:
model_name = "allenai/scibert_scivocab_uncased"
metadata = get_precomputed_doc_embeddings(model_name)


Precomputing document embeddings.


100%|██████████| 7718/7718 [03:58<00:00, 32.32it/s]


In [56]:
match_model = MatchPyramid()

train_dataset = MatchPyramidTripletDataset(df_query_train, metadata, num_negatives=1)


train_matchpyramid(match_model, train_dataset, tokenizer,scibert, metadata)


Epoch 1/6:  76%|███████▌  | 921/1216 [06:04<01:44,  2.83it/s]

Error at batch 921, sample 3: nan


Epoch 1/6:  94%|█████████▍| 1148/1216 [07:34<00:26,  2.56it/s]

Skipped small embedding (idx 6)


 Epoch 1/6 — Total Loss: 280.0087


...

Epoch 6/6:  42%|████▏     | 507/1216 [03:24<04:17,  2.75it/s]

Skipped small embedding (idx 2)


Epoch 6/6:  74%|███████▍  | 904/1216 [06:08<02:13,  2.33it/s]

Error at batch 904, sample 4: nan


 Epoch 6/6 — Total Loss: 170.4016


In [57]:
torch.save(match_model.state_dict(), "matchpyramid_scibert.pth")


In [58]:
from tqdm import tqdm

def rerank_with_matchpyramid(model, df_query, tokenizer, transformer_model, metadata, top_k=5):
    model.eval()
    DEVICE = next(model.parameters()).device
    reranked = []

    for idx, row in tqdm(df_query.iterrows(), total=len(df_query), desc=" Reranking Queries"):
        query = row["tweet_text"]
        candidates = row["bm25_topk"]
        scores = []

        try:
            q_inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=100).to(DEVICE)
            q_embs = transformer_model(**q_inputs).last_hidden_state.squeeze(0)
            q_mask = q_inputs['attention_mask'].squeeze(0).bool()
            q_embs = q_embs[q_mask]
        except:
            reranked.append([])
            continue

        for doc_id in candidates:
            try:
                doc_emb = torch.load(metadata[doc_id]["path"]).to(DEVICE)[:100]
                if q_embs.size(0) == 0 or doc_emb.size(0) == 0:
                    continue

                match_matrix = create_match_matrix(q_embs, doc_emb).unsqueeze(0)
                if torch.isnan(match_matrix).any():
                    continue

                score = model(match_matrix).item()
                scores.append((doc_id, score))
            except:
                continue

        sorted_docs = sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]
        reranked.append([doc_id for doc_id, _ in sorted_docs])

    df_query["reranked_topk"] = reranked
    return df_query


In [59]:
df_query_dev_reranked = rerank_with_matchpyramid(
    match_model, df_query_dev, tokenizer, scibert, metadata, top_k=5
)


 Reranking Queries: 100%|██████████| 1400/1400 [22:59<00:00,  1.02it/s]


In [60]:
def compute_mrr(df, truth_column="cord_uid", preds_column="reranked_topk"):
    reciprocal_ranks = []

    for _, row in df.iterrows():
        true_doc = row[truth_column]
        predicted_list = row[preds_column]

        if true_doc in predicted_list:
            rank = predicted_list.index(true_doc) + 1
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)

    return sum(reciprocal_ranks) / len(reciprocal_ranks)


mrr_score = compute_mrr(df_query_dev_reranked)
print(f" MRR Score: {mrr_score:.4f}")


 MRR Score: 0.4201
